<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_10_08_Semantic_Search_from_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a PDF from a URL, and search for relevant text chunks.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


Initialize Graphlit

In [3]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [4]:
async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def query_similar_contents_by_text(query_text: str):
    if graphlit.client is None:
        return

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                searchType=enums.SearchTypes.VECTOR,
                search=query_text,
                limit=10
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [6]:
from IPython.display import display, Markdown

# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

search_text = 'large language model'

content_id = await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Attention%20Is%20All%20You%20Need.1706.03762.pdf")

if content_id is not None:
    print(f'Ingested content [{content_id}].')

    contents = await query_similar_contents_by_text(search_text)

    if contents is not None:
        for content in contents:
            if content is not None and content.pages is not None:
                for page in content.pages:
                    if page is not None and page.chunks is not None:
                        display(Markdown(f"### Page [{page.index}]: {len(page.chunks)} chunk(s)"))

                        for chunk in page.chunks:
                            if chunk is not None:
                                display(Markdown(f"#### Chunk [{chunk.index}]: Relevance [{chunk.relevance}]"))
                                display(Markdown(chunk.text))

Deleted all contents.
Ingested content [0e63aca7-62de-4071-8e76-fc7cf8b23ce9]:


### Page [0]: 1 chunk(s)

#### Chunk [0]: Relevance [0.80147076]

Attention Is All You Need
| Ashish Vaswani* Google Brain | Noam Shazeer* Google Brain | Niki Parmar* Google Research | Jakob Uszkoreit* Google Research |
| --- | --- | --- | --- |
| avaswani@google.com | noam@google.com | nikip@google.com | usz@google.com |

| Llion Jones* Google Research | Aidan N. Gomez* + University of Toronto | Łukasz Kaiser* Google Brain |
| --- | --- | --- |
| llion@google.com | aidan@cs.toronto.edu | lukaszkaiser@google.com |

Illia Polosukhin* * illia.polosukhin@gmail.com

### Page [1]: 1 chunk(s)

...

### Page [24]: 2 chunk(s)

#### Chunk [0]: Relevance [0.8117264]

perfect
,
, but
,
but its
but
, but
its
its
its
application
application should
application should
application should
should
be
be
be
be
just
just
just
just
-
-
-
-
this
this
this
this
is
is
is
is
what
what
what
what
we
we
we
we
are
are
are
are
missing
missing
missing
missing
,
,
,
in my
, in
in
in
my
my
my
opinion
opinion
opinion
opinion
.
.
.
.
<EOS>
<EOS>
<EOS>
<EOS>
<pad>
<pad>
<pad>
<pad>
The
The
Law
Law
will
will
never
never
be
be
perfect
perfect
,
,
but
but
its
its
application should
application should
be
be
just
just
-
-
this
this
is
is
what
what
we
we
are
are
missing
missing
, in
,
in
my
my opinion
opinion
.
.
<EOS>
<EOS>
<pad>
<pad>
The
The
Law
Law
will
will
never
never
be
be
perfect
perfect
, but
,
but
its
its
application
application
should
should
be
be
just
just
-
-
this
this
is
is
what
what
we
we
are
are
missing
missing
,
,
in
in
my
my
opinion
opinion
.
.
<EOS>
<EOS>
<pad>
<pad>
Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the
sentence. We give two such examples above, from two different heads from the encoder self-attention at layer 5 of 6. The heads clearly learned to perform different tasks.

#### Chunk [1]: Relevance [0.80714697]

Attention Visualizations
the word 'making'. Different colors represent different heads. Best viewed in color.
Figure 3: An example of the attention mechanism following long-distance dependencies in the encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of the verb 'making', completing the phrase 'making ... more difficult'. Attentions here shown only for
It
It
is
is
in
in
this
this
spirit
spirit
that
that
a
a
majority
majority
of American
of
American
governments
governments
have passed
have
passed
new
new
laws
laws
since
since
2009
2009
making
making
the registration
the registration or voting process
or voting
process
more
more
difficult
difficult
.
.
<EOS>
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
and 6. Note that the attentions are very sharp for this word.
Full attentions for head 5. Bottom: Isolated attentions from just the word 'its' for attention heads 5 Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top:
The
The Law
The
The
Law
Law
Law
will
will
will
will
never be perfect
never
never
never
be
be
be
perfect
perfect perfect
,
, but
,
but its
but
, but
its
its
its
application
application should
application should
application should
should
be
be
be
be
just
just
just
just
-
-
-
-
this
this
this
this
is
is
is
is
what
what
what
what
we
we
we
we
are
are
are
are
missing
missing
missing
missing
,
,
,
in my
, in
in
